<a href="https://colab.research.google.com/github/dharmesh312/CVML-COURSE/blob/master/steganylysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import tensorflow
import tensorflow as tf
from numpy import *
from PIL import Image
from keras.callbacks import TensorBoard
import cv2
from tqdm import tqdm
from sklearn.utils import shuffle


In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip3 install image_slicer

In [0]:
# slicing the images from 512*512 to 4 256*256 image
import os, os.path
import image_slicer
import shutil

op_path = 'drive/BOSSbase_1.01'

imlist=os.listdir(op_path)  
imlist.sort()
im1=array(Image.open(op_path + '/' + imlist[0]))

tiles = ()
for i in range(len(imlist)):
  tile = image_slicer.slice(op_path + '/' + imlist[i], 4, save=False)
  image_slicer.save_tiles(tile, directory='drive/croppedImages/' ,prefix='cover.' + str(i))

m,n=im1.shape[0:2]
print (m,n)



In [0]:
!pip install tensorboardcolab



In [0]:
from tensorboardcolab import TensorBoardColab
tbc = TensorBoardColab()
summary_writer = tbc.get_writer()


In [0]:
import os, os.path
trainDir = "drive/Bbase/coverFinal/"
from sklearn.preprocessing import normalize

def normalize(nparray,alpha=-1):

    # Normalization of our first convolutional layer

    nparray = np.array(nparray)
    sumation = nparray.sum()
    nparray = nparray/sumation
    nparray = np.array(nparray) # return a nparray
    return nparray


def label_image(img):
  word = img.split(".")[0]
  if word == "stego":
    return [0,1]
  elif word == "cover":
    return [1,0]

def create_train_data():
    training_data = []

    i  = 0
    arr = os.listdir(trainDir)
    random.shuffle(arr)
    for img in tqdm(arr[:3000]):
        label = label_image(img)
        path = os.path.join(trainDir,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        training_data.append( [normalize(np.array(img),-1) , np.array(label)])

#     np.save('drive/train_data.npy', training_data)
#     random.shuffle(training_data)
    print ("\n The shape of the images is:")
    return training_data

trainData = create_train_data()
validationData = trainData[2600:2800]
trainData1 = trainData[:2600]
testDataEntire = trainData[2800:]
trainDataImages = np.array([i[0] for i in trainData1]).reshape([-1,256,256,1])  
trainDataLabels = np.array([i[1] for i in trainData1])
validData = np.array([i[0] for i in validationData]).reshape([-1,256,256,1])  
validLabels = np.array([i[1] for i in validationData])
testDataImages = np.array([i[0] for i in testDataEntire]).reshape([-1,256,256,1])  
testDataLabels = np.array([i[1] for i in testDataEntire])



In [0]:
# architecture

from tensorflow.contrib.layers import flatten

def residual_unit(layer , inputSize , outputSize ):
      convLayerResNetBlock1 = {"Weights" : tf.Variable(tf.truncated_normal([3,3,inputSize, outputSize] , mean = 0 ,stddev = 0.1))  , "Biases"  :  tf.Variable(tf.zeros(inputSize)) }
      convLayerResNetBlock2 = {"Weights" : tf.Variable(tf.truncated_normal([3,3,inputSize, outputSize] , mean = 0 ,stddev = 0.1))  , "Biases"  :  tf.Variable(tf.zeros(inputSize)) }
      
      stepL1 = tf.nn.conv2d(layer , convLayerResNetBlock1["Weights"] , strides = [1,1,1,1] , padding = 'SAME') + convLayerResNetBlock1['Biases']
      stepL1 = tf.nn.relu(stepL1)
      stepL1 = tf.nn.max_pool(stepL1, ksize = [1,2,2,1], strides = [1,1,1,1], padding = 'SAME')
      stepL1 = tf.layers.batch_normalization(stepL1)

      stepL2 = tf.nn.conv2d(stepL1 , convLayerResNetBlock2["Weights"] , strides = [1,1,1,1] , padding = 'SAME') + convLayerResNetBlock2['Biases']
      stepL2 = tf.nn.relu(stepL2)
      stepL2 = tf.nn.max_pool(stepL2, ksize = [1,2,2,1], strides = [1,1,1,1], padding = 'SAME')
      stepL2 = tf.layers.batch_normalization(stepL2)
      
      return layer + stepL2
      
      
      
def convNet(image):
      layerInResBlock = 1
      convLayerRes = {"Weights" : tf.Variable(tf.truncated_normal([5,5,1,12]  , mean = 0 ,stddev = 0.1))}
      convLayer1 = {"Weights" : tf.Variable(tf.truncated_normal([7,7,12,64] , mean = 0 ,stddev = 0.1))  , "Biases"  :  tf.Variable(tf.zeros(64)) } 
      dimensionIncreaseLayer1 = {"Weights" : tf.Variable(tf.truncated_normal([3,3,64,128] , mean = 0 ,stddev = 0.1))  , "Biases"  :  tf.Variable(tf.zeros(128)) } 
      dimensionIncreaseLayer2 = {"Weights" : tf.Variable(tf.truncated_normal([3,3,128,256] , mean = 0 ,stddev = 0.1))  , "Biases"  :  tf.Variable(tf.zeros(256)) } 
      dimensionIncreaseLayer3 = {"Weights" : tf.Variable(tf.truncated_normal([3,3,256,512] , mean = 0 ,stddev = 0.1))  , "Biases"  :  tf.Variable(tf.zeros(512)) } 
      outputLayer = {"Weights" : tf.Variable(tf.truncated_normal( [131072,2]  , mean = 0 ,stddev = 0.1 ))  , "Biases"  :  tf.Variable(tf.zeros(2)) }
      
#       ip 256*256*1 ,op = 256*256*12 // special prediction layer
      convRes = tf.nn.conv2d(image , convLayerRes["Weights"] , strides = [1,1,1,1] , padding = 'SAME') 
  
#      ip = 256*256*12 op = 128*128*64 // first conv layer from paper
      conv1 = tf.nn.conv2d(convRes , convLayer1["Weights"] , strides = [1,1,1,1] , padding = 'SAME') + convLayer1['Biases']
      activation1 = tf.nn.relu(conv1)
      pool1 = tf.nn.max_pool(activation1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
      layer = tf.layers.batch_normalization(pool1)
      
#       ResNet Part Starts // n1 set of layers ip 128*128*64 = 64*64*128
      for i in range(layerInResBlock):
          layer = residual_unit(layer , 64 , 64)
#     downsampling  
      layer = tf.nn.conv2d(layer , dimensionIncreaseLayer1["Weights"] , strides = [1,2,2,1] , padding = 'SAME') + dimensionIncreaseLayer1['Biases']
      layer = tf.nn.relu(layer)
      layer = tf.layers.batch_normalization(layer)

#     n2 set of blocks 64*64*128 -> 32*32*256
      for i in range(layerInResBlock):
          layer = residual_unit(layer , 128 , 128)
#     downsampling
      layer = tf.nn.conv2d(layer , dimensionIncreaseLayer2["Weights"] , strides = [1,2,2,1] , padding = 'SAME') + dimensionIncreaseLayer2['Biases']
      layer = tf.nn.relu(layer)
      layer = tf.layers.batch_normalization(layer)
    
    
#     n3 set of blocks 32*32*256 -> 16*16*512
      for i in range(layerInResBlock):
          layer = residual_unit(layer , 256 , 256)
#     downsampling
      layer = tf.nn.conv2d(layer , dimensionIncreaseLayer3["Weights"] , strides = [1,2,2,1] , padding = 'SAME') + dimensionIncreaseLayer3['Biases']
      layer = tf.nn.relu(layer)
      layer = tf.layers.batch_normalization(layer)      

#     n4 set of blocks  16*16*512 - > 131072    
      for i in range(layerInResBlock):
          layer = residual_unit(layer , 512 , 512)
      
      layer = tf.nn.max_pool(layer, ksize = [1,2,2,1], strides = [1,1,1,1], padding = 'SAME')
      fcc = flatten(layer)
      
      output = tf.add(tf.matmul(fcc , outputLayer['Weights']) , outputLayer['Biases'])


      return output

In [0]:
epochs = 20
batch_size = 10
x = tf.placeholder(tf.float32, (None, 256, 256, 1))
y = tf.placeholder(tf.float32, shape=[None, 2])
rate = 0.001

In [0]:

print (tf.test.gpu_device_name())
from tensorflow.python.client import device_lib
print (device_lib.list_local_devices())

In [0]:
from sklearn.utils import shuffle # mixing up or currently ordered data that might lead our network astray in training.

def normalizeMykernel(nparray,alpha=-1):

    # Normalization of our first convolutional layer

    nparray = np.array(nparray)
    nparray[2,2] = alpha # Evaluate the influence of this alpha
    nparray = np.ma.array(nparray, mask=False)
    nparray.mask[2,2] = True  #Mask the center so he will not appear in the normalisation
    sumation = nparray.sum()
    nparray = nparray/sumation
    nparray = np.array(nparray) # return a nparray
    return nparray


def trainNetwork(x):
#   print (prediction.shape , oneHotY.shape)
  prediction = convNet(x)
  cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits( logits = prediction, labels = y) )
  optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)
  
  correct = tf.equal(tf.argmax(prediction, 1) , tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct ,'float'))
  
  print ("Starting training ...." )
  #taining the model
  with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for i in range(epochs):
      train_data_image, train_data_label = shuffle(trainDataImages, trainDataLabels , random_state = 0)
      loss = 0
      
#       initialiing the new values for the first vconv layer kernel according to the plan from the texas paper 
      gr = tf.get_default_graph()
      tf.summary.FileWriter(gr)
      conv1_kernel_val = gr.get_tensor_by_name('Variable:0').eval()      
      normalizeMykernel(conv1_kernel_val,-1)
      conv1_kernel_val = normalizeMykernel(conv1_kernel_val,-1)
      sess.run(tf.assign( gr.get_tensor_by_name('Variable:0') , conv1_kernel_val))
      
      for start in range(0 , len(train_data_image) , batch_size):
        last = start + batch_size
        batch_x, batch_y = train_data_image[ start : last ], train_data_label[ start : last ]
        c, l = sess.run([optimizer,cost] ,feed_dict = { x:batch_x , y:batch_y })
        loss += l
      
      tbc.save_value("Loss per Epoch", "xyc", i, loss)

      print ("Epoch:" , i , " with Loss :" , loss) 
      
      print ("Validation Set accuracy is :", accuracy.eval({x : validData , y : validLabels}))
      tbc.save_value(" validation accuracy", "asdasd", i, accuracy.eval({x : validData , y : validLabels}))
#       print (validLabels , sess.run(prediction,feed_dict = {x : validData}))
      
      
    print ("Test Set accuracy is :", accuracy.eval({x : testDataImages , y : testDataLabels}))

In [0]:
trainNetwork(x)